In [34]:
import pandas as pd
import numpy as np

In [35]:
x_train = pd.read_csv('WaterPumpData/features_train.csv')
y_train = pd.read_csv('WaterPumpData/labels_train.csv')

In [36]:
y_train['label'] = y_train.status_group.apply(lambda x: 1 if x == 'functional' else 0)

In [37]:
funder = x_train.funder.str.lower().value_counts()
funder = funder[funder > 250]
x_train['funder_mod'] = [i if i in funder else 'Unknown' for i in x_train.funder]

installer = x_train.funder.str.lower().value_counts()
installer = funder[funder > 250]
installer
x_train['installer_mod'] = [i if i in installer else 'Unknown' for i in x_train.installer]

In [38]:
import random
from random import sample

validation_set = sample(list(x_train.id), len(x_train)//5)

In [39]:
xx_train = x_train.loc[~x_train.id.isin(validation_set)]
xx_validate = x_train.loc[x_train.id.isin(validation_set)]
yy_train = y_train.loc[~y_train.id.isin(validation_set)].label
yy_validate = y_train.loc[y_train.id.isin(validation_set)].label

In [40]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

def pipeline(feature_set, numeric_set):
    numeric_features = numeric_set
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())])

    categorical_features =  [i for i in feature_set if i not in numeric_set]
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features),
            ('num', numeric_transformer, numeric_features)
        ])

    clf = Pipeline(steps=[('preprocessor', preprocessor)
                         ])

    clf.fit(xx_train)
    
    return clf

In [49]:
feature_set_7 = ['amount_tsh', 'population', 'funder_mod', 'district_code', 'gps_height', 'region', 'public_meeting', 'scheme_management', 'permit', 'construction_year', 'extraction_type_group', 'management_group', 'payment_type', 'quality_group', 'quantity_group', 'source_type', 'waterpoint_type_group']
numeric_set = ['amount_tsh', 'population', 'gps_height', 'construction_year']

clf = pipeline(feature_set_7, numeric_set)

xxx_train = pd.DataFrame(clf.transform(xx_train).todense())
xxx_validate = pd.DataFrame(clf.transform(xx_validate).todense())
yyy_train = pd.DataFrame(yy_train)
yyy_validate = pd.DataFrame(yy_validate)

In [50]:
from keras.models import Sequential, Model
from keras.layers import Dense, concatenate, Input
from tensorflow.keras.optimizers import Adagrad, Adam
from keras.callbacks import EarlyStopping

In [51]:
ip = Input(shape=xxx_train.shape[1], name='ip')
x1 = Dense(100, activation='relu')(ip)
x2 = Dense(50, activation='relu')(x1)
x3 = Dense(10, activation='relu')(x2)
output = Dense(1, activation='sigmoid')(x3)

model = Model(inputs = ip, outputs=output)
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [52]:
model.fit(xxx_train, yyy_train, epochs=25, batch_size = 16)

Epoch 1/25
2970/2970 [==============================] - 3s 769us/step - loss: 0.4939 - accuracy: 0.7524
Epoch 2/25
2970/2970 [==============================] - 2s 840us/step - loss: 0.4542 - accuracy: 0.7788
Epoch 3/25
2970/2970 [==============================] - 2s 814us/step - loss: 0.4373 - accuracy: 0.7892
Epoch 4/25
2970/2970 [==============================] - 3s 857us/step - loss: 0.4272 - accuracy: 0.7961
Epoch 5/25
2970/2970 [==============================] - 2s 828us/step - loss: 0.4173 - accuracy: 0.8009
Epoch 6/25
2970/2970 [==============================] - 2s 814us/step - loss: 0.4111 - accuracy: 0.8044
Epoch 7/25
2970/2970 [==============================] - 2s 830us/step - loss: 0.4049 - accuracy: 0.8079
Epoch 8/25
2970/2970 [==============================] - 3s 886us/step - loss: 0.3992 - accuracy: 0.8127
Epoch 9/25
2970/2970 [==============================] - 3s 891us/step - loss: 0.3953 - accuracy: 0.8130
Epoch 10/25
2970/2970 [==============================] - 2s 802u

In [53]:
pred_raw = model.predict(xxx_validate).flatten()
pred = [round(i) for i in pred_raw]
yyy_validate['pred'] = pred
yyy_validate['correct'] = (yyy_validate.label == yyy_validate.pred)

yyy_validate.correct.value_counts()

True     9396
False    2484
Name: correct, dtype: int64

In [54]:
feature_set_8 = ['amount_tsh', 'population', 'funder_mod', 'lga', 'scheme_management', 'permit', 'construction_year', 'extraction_type_group', 'management_group', 'payment', 'quality_group', 'quantity_group', 'source_type', 'waterpoint_type_group']
numeric_features = ['amount_tsh', 'population', 'construction_year']

clf = pipeline(feature_set_8, numeric_features)

In [55]:
xxx_train = pd.DataFrame(clf.transform(xx_train).todense())
xxx_validate = pd.DataFrame(clf.transform(xx_validate).todense())
yyy_train = pd.DataFrame(yy_train)
yyy_validate = pd.DataFrame(yy_validate)

In [59]:
ip = Input(shape=xxx_train.shape[1], name='ip')
x2 = Dense(100, activation='elu')(ip)
x3 = Dense(100, activation='elu')(x2)
x4 = Dense(20, activation='relu')(x3)
output = Dense(1, activation='sigmoid')(x4)

model = Model(inputs = ip, outputs=output)
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])
model.fit(xxx_train, yyy_train, epochs=25, batch_size = 16)

Epoch 1/25
2970/2970 [==============================] - 3s 887us/step - loss: 0.4882 - accuracy: 0.7541
Epoch 2/25
2970/2970 [==============================] - 2s 838us/step - loss: 0.4569 - accuracy: 0.7763
Epoch 3/25
2970/2970 [==============================] - 2s 819us/step - loss: 0.4462 - accuracy: 0.7823
Epoch 4/25
2970/2970 [==============================] - 3s 884us/step - loss: 0.4398 - accuracy: 0.7873
Epoch 5/25
2970/2970 [==============================] - 2s 794us/step - loss: 0.4334 - accuracy: 0.7911
Epoch 6/25
2970/2970 [==============================] - 2s 817us/step - loss: 0.4287 - accuracy: 0.7936
Epoch 7/25
2970/2970 [==============================] - 3s 928us/step - loss: 0.4256 - accuracy: 0.79530s - loss: 0
Epoch 8/25
2970/2970 [==============================] - 2s 840us/step - loss: 0.4217 - accuracy: 0.7965
Epoch 9/25
2970/2970 [==============================] - 2s 825us/step - loss: 0.4181 - accuracy: 0.7982
Epoch 10/25
2970/2970 [=============================

In [60]:
pred_raw = model.predict(xxx_validate).flatten()
pred = [round(i) for i in pred_raw]
yyy_validate['pred'] = pred
yyy_validate['correct'] = (yyy_validate.label == yyy_validate.pred)

yyy_validate.correct.value_counts()

True     9379
False    2501
Name: correct, dtype: int64

In [61]:
ip = Input(shape=xxx_train.shape[1], name='ip')
x1 = Dense(250, activation='relu')(ip)
x2 = Dense(100, activation='relu')(x1)
x3 = Dense(50, activation='relu')(x2)
x4 = Dense(10, activation='relu')(x3)
output = Dense(1, activation='sigmoid')(x4)

model = Model(inputs = ip, outputs=output)
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])
model.fit(xxx_train, yyy_train, epochs=25, batch_size = 16)

Epoch 1/25
2970/2970 [==============================] - 4s 1ms/step - loss: 0.4792 - accuracy: 0.7629
Epoch 2/25
2970/2970 [==============================] - 3s 1ms/step - loss: 0.4388 - accuracy: 0.7891
Epoch 3/25
2970/2970 [==============================] - 3s 977us/step - loss: 0.4240 - accuracy: 0.7961
Epoch 4/25
2970/2970 [==============================] - 3s 970us/step - loss: 0.4128 - accuracy: 0.8033
Epoch 5/25
2970/2970 [==============================] - 3s 992us/step - loss: 0.4042 - accuracy: 0.8084
Epoch 6/25
2970/2970 [==============================] - 3s 1ms/step - loss: 0.3965 - accuracy: 0.8124
Epoch 7/25
2970/2970 [==============================] - 3s 1ms/step - loss: 0.3903 - accuracy: 0.8155
Epoch 8/25
2970/2970 [==============================] - 3s 1ms/step - loss: 0.3842 - accuracy: 0.8190
Epoch 9/25
2970/2970 [==============================] - 3s 1ms/step - loss: 0.3799 - accuracy: 0.8198
Epoch 10/25
2970/2970 [==============================] - 3s 1ms/step - loss:

In [62]:
pred_raw = model.predict(xxx_validate).flatten()
pred = [round(i) for i in pred_raw]
yyy_validate['pred'] = pred
yyy_validate['correct'] = (yyy_validate.label == yyy_validate.pred)

yyy_validate.correct.value_counts()

True     9388
False    2492
Name: correct, dtype: int64